### a. Loading the dataset

In [1]:
import pandas as pd

In [2]:
from hazm import Normalizer, Lemmatizer,word_tokenize, stopwords_list, Stemmer

In [3]:
data =pd.read_csv("emails.csv")

In [4]:
data.head()

,text,label
0,﻿ممنون آقا سامان.\nمن پارسال اصلا آزاد شرکت نک...,ham
1,﻿سلام آقای کریمی\nبالاخره آزمونارشد تموم شد من...,ham
2,﻿درود بر حاج وحیدی بنده بعنوان یک دکتری تاریخ ...,ham
3,﻿با سلام و احترام\nضمن تقدیر از مسولین محترم ...,ham
4,﻿با سلام اینجانب یک دستگاه خودرو پراید 131 با ...,ham


## b. Preproccessing 

### Normalization 

In [5]:
normalizer = Normalizer()

data['text'] = data['text'].apply(normalizer.normalize)
data['text'] = data['text']. apply(normalizer.correct_spacing)
data['text'] = data['text']. apply(normalizer.remove_diacritics)
data['text'] = data['text']. apply(normalizer.remove_specials_chars)
data['text'] = data['text']. apply(normalizer.decrease_repeated_chars)
data['text'] = data['text']. apply(normalizer.persian_number)
    

In [6]:
stop_words=stopwords_list()
lemmatizer = Lemmatizer()

def tokenization(text): 
    tokens = [i for i in word_tokenize(text) if i not in stop_words ]
    tokens=[i for i in tokens if i.isalnum()]
    tokens = [lemmatizer.lemmatize(i) for i in tokens]

    text = ' '.join(tokens)

    
    return text

In [7]:
data['text'] = data['text'].apply(tokenization)

In [8]:
data.head()

,text,label
0,آقا سامان پارسال اصلا آزاد شرکت سراسر قبول شد#...,ham
1,کریمی بالاخره آزمونارشد تموم راحت شد#شو یکم را...,ham
2,حاج وحید بنده بعنوان دکتری تاریخ دستی تاریخ مح...,ham
3,سلام احترام تقدیر مسولین محترم سایت تابناک رسا...,ham
4,سلام اینجانب دستگاه خودرو پراید ۱۳۱ شماره درخو...,ham


## c. Splitting and Scaling 

In [9]:
from sklearn.model_selection import train_test_split

## Bag of words representation  

#### counting the number of occurances of each word in the dataset

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(data.text)


CountVectorizer()

In [11]:
print(vect.vocabulary_)

{'آقا': 3022, 'سامان': 9382, 'پارسال': 15231, 'اصلا': 3911, 'آزاد': 2933, 'شرکت': 10058, 'سراسر': 9485, 'قبول': 11437, 'شد': 10015, 'شو': 10216, 'بدون': 5193, 'شرایط': 10029, 'چطوریه': 15838, 'واسه': 14644, 'سال': 9358, 'مرسی': 12464, 'راهنمایی': 8667, 'هاتون': 14225, 'کریمی': 16225, 'بالاخره': 4976, 'آزمونارشد': 2950, 'تموم': 6539, 'راحت': 8603, 'یکم': 16994, 'تخصص': 6194, 'بتون': 5077, 'اشتراک': 3867, 'گرفت': 16644, 'گیر': 16844, 'کدومه': 16178, 'جامعه': 6838, 'شناخت': 10149, 'شناس': 10153, 'روانشناسی': 8878, 'زبان': 9101, 'کتابای': 16134, 'مرجع': 12437, 'لطفا': 11724, 'بهم': 5718, 'معرفی': 12822, 'صرفا': 10404, 'خاطر': 7389, 'خودم': 7685, 'کنکور': 16411, 'دوستان': 8401, 'دیگه': 8559, 'اگه': 4626, 'کمک': 16366, 'ممنونمی': 12996, 'شم': 10115, 'حاج': 7116, 'وحید': 14826, 'بنده': 5646, 'بعنوان': 5535, 'دکتری': 8469, 'تاریخ': 6037, 'دستی': 8213, 'محلی': 12259, 'داشت': 7844, 'دار': 7812, 'موارد': 13171, 'ذکر': 8575, 'توسط': 6650, 'پزو': 15499, 'هشگر': 14360, 'اسناد': 3815, 'تاریخی': 6040,

In [12]:
bag_of_words = vect.transform(data.text)
bag_of_words

<1000x18805 sparse matrix of type '<class 'numpy.int64'>'
	with 102871 stored elements in Compressed Sparse Row format>

In [13]:
X = bag_of_words.toarray()

In [14]:
Y = data.label

In [15]:
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.3,random_state=30)

## d. training a KNN classifier model 

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
accuracies = []
for k in range(1,21):

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    prediction = knn.predict(X_test)

    acc = accuracy_score(y_test, prediction)
    accuracies.append({k, acc})


In [51]:
max_acc = list(max((accuracies)))[0] * 100
best_k = list(max((accuracies)))[1]


In [57]:
for k in list(accuracies): 
    acc = round(list(k)[0], 2)    
    k = list(k)[1]
    print('k: {}   accuracy : {}'.format(k, acc))

k: 1   accuracy : 0.88
k: 2   accuracy : 0.84
k: 3   accuracy : 0.87
k: 4   accuracy : 0.84
k: 5   accuracy : 0.85
k: 6   accuracy : 0.83
k: 7   accuracy : 0.84
k: 0.8166666666666667   accuracy : 8
k: 9   accuracy : 0.82
k: 10   accuracy : 0.8
k: 11   accuracy : 0.81
k: 12   accuracy : 0.79
k: 13   accuracy : 0.8
k: 14   accuracy : 0.78
k: 15   accuracy : 0.78
k: 0.77   accuracy : 16
k: 17   accuracy : 0.77
k: 18   accuracy : 0.77
k: 19   accuracy : 0.77
k: 20   accuracy : 0.76


In [45]:
print('the best k is {} and the best accuracy is {} percent!'.format(best_k, round(max_acc, 2)))

the best k is 1 and the best accuracy is 88.0 percent!


### the best k is 1 and its accuracy score is almost 89 percent!